<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM1110-1 Workshop 4: Nonlinear FEM with plasticity (notebook 3)

In [ ]:
import sys
sys.path.append('../../')

import numpy as np
import matplotlib.pyplot as plt
import contextlib

from utils import proputils as pu
from names import GlobNames as gn
import main
import declare

%matplotlib widget

## Plasticity for a more complex 2D domain

In this final demonstration, we look at the following geometry:

<center><img src="https://raw.githubusercontent.com/ibcmrocha/public/main/multihole.png" alt="mesh" width="300"/></center>

with Dirichlet BCs similar to the ones from the previous notebook, i.e. we pull this square-shaped domain on the right edge while keeping the left edge fixed horizontally and the bottom edge fixed vertically.

We set up a first version for this model in `voids.pro`. We run a displacement-controlled simulation for $40$ time steps with `props['model']['dispcontrol']['timeSignal'] = 't'`, which means the displacement at the right edge of the model is **monotonically increasing**. Run the simulation below as it is:

In [ ]:
props = pu.parse_file('voids.pro')

globdat = main.jive(props)

From `J2Material`, we store in `globdat` the maximum value of $\kappa$ throughout the whole domain $\Omega$ after every time step. We can then plot this to see how $\kappa$ is evolving in time:

In [ ]:
plt.figure()
plt.xlabel('Time step')
plt.ylabel('$\max_\Omega(\kappa)$')
plt.plot(range(len(globdat['maxkappa'])),globdat['maxkappa'])

For this model, $\kappa$ is an accumulated measure of plastic strain given by:

$$\kappa(t)=\displaystyle\int_0^t\sqrt{\frac{2}{3}\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t):\dot{\boldsymbol{\varepsilon}}^\mathrm{p}(t)}\,dt$$

With this in mind, do the results above make sense?

Finally, we can look at the displacement field evolution for this model:

In [ ]:
view = globdat[gn.MODULEFACTORY].get_module('View','view')

props['view'] = {}
props['view']['plot'] = 'solution[dx]'
props['view']['deform'] = 1
props['view']['ncolors'] = 100
props['view']['interactive'] = 'True'
props['view']['colorMap'] = 'plasma_r'

view.init(props, globdat)
status = view.shutdown(globdat)

## Unloading behavior

Let us now run the same model, but change `props['model']['dispcontrol']['timeSignal']` to include loading/unloading/reloading branches. More specifically, try to make the time signal reflect the following plot:

<center><img src="https://raw.githubusercontent.com/ibcmrocha/public/main/pwlinearsignal.png" alt="mesh" width="500"/></center>

To do that, you can use the `np.where()` function. For instance, to create a single loading/unloading branch we could write

`props['model']['dispcontrol']['timeSignal'] = 'np.where(t<t1, t, 2*t1-t)'`

where `t1` is the switching point. You can read this `np.where()` call as being equivalent to:

```
if t < t1:
    signal = t
else:
    signal = 2.*t1 - t
```

Now try to reproduce the curve above. For that you will need to <span style="color:red">nest a second `np.where()` call inside the first one</span>.

In [ ]:
props = pu.parse_file('voids.pro')

# props['model']['dispcontrol']['timeSignal'] = '???'
props['nonlin']['nsteps'] = 70

globdat = main.jive(props)

Now let us look at the evolution of $\kappa$ again:

In [ ]:
plt.figure()
plt.xlabel('Time step')
plt.ylabel('$\max_\Omega(\kappa)$')
plt.plot(range(len(globdat['maxkappa'])),globdat['maxkappa'])

Can you make sense of what you see?

## More complex time signals

As one final example, we run the model for the following more complex Dirichlet BC time signal:

<center><img src="https://raw.githubusercontent.com/ibcmrocha/public/main/sinesignal.png" alt="mesh" width="500"/></center>

given by the following expression:

$$\mathrm{signal} = 0.8t-0.2t\sin\left(\frac{2\pi t}{10}\right)$$

and now we run the simulation for $100$ steps and reduce the time step size to guarantee stable convergence. Run the model below:

In [ ]:
props = pu.parse_file('voids.pro')

# props['model']['dispcontrol']['timeSignal'] = '???'
props['model']['dispcontrol']['deltaTime'] = 0.5
props['nonlin']['nsteps'] = 100

globdat = main.jive(props)

We can then take a look at how $\kappa$ evolves:

In [ ]:
plt.figure()
plt.xlabel('Time step')
plt.ylabel('$\max_\Omega(\kappa)$')
plt.plot(range(len(globdat['maxkappa'])),globdat['maxkappa'])

And also plot the deformations with an interactive plot:

In [ ]:
view = globdat[gn.MODULEFACTORY].get_module('View','view')

props['view'] = {}
props['view']['plot'] = 'solution[dx]'
props['view']['deform'] = 1
props['view']['ncolors'] = 100
props['view']['interactive'] = 'True'
props['view']['colorMap'] = 'plasma_r'

view.init(props, globdat)
status = view.shutdown(globdat)